In [ ]:
dbutils.widgets.dropdown('src_type', 'sqlserver', ['sqlserver', 'postgresql', 'delta'], '01 Source Type')
dbutils.widgets.text('src_catalog', '', '02 Source Catalog')
dbutils.widgets.text('src_schema', '', '03 Source Schema')
dbutils.widgets.text('src_table', '', '04 Source Table')
dbutils.widgets.text('jdbc_config_file', '', '05 Source Table')
dbutils.widgets.text('partition_col', '', '06 Source Partition Column')
dbutils.widgets.text('partition_size_mb', '', '07 Partition Size MB')
dbutils.widgets.text('tgt_catalog', '', '08 Target Catalog')
dbutils.widgets.text('tgt_schema', '', '09 Target Schema')
dbutils.widgets.text('tgt_table', '', '10 Target Table')

In [ ]:
from lakefed_ingest.main import *

In [ ]:
src_type = dbutils.widgets.get('src_type')
src_catalog = dbutils.widgets.get('src_catalog')
src_schema = dbutils.widgets.get('src_schema')
src_table = dbutils.widgets.get('src_table')
jdbc_config_file = dbutils.widgets.get('jdbc_config_file')
partition_col = dbutils.widgets.get('partition_col')
partition_size_mb = int(dbutils.widgets.get('partition_size_mb'))
tgt_catalog = dbutils.widgets.get('tgt_catalog')
tgt_schema = dbutils.widgets.get('tgt_schema')
tgt_table = dbutils.widgets.get('tgt_table')

jdbc_config_file = None if jdbc_config_file == '' else jdbc_config_file

In [ ]:
table_size_mb = 0

if src_type == 'sqlserver':
    table_size_mb = get_table_size_sqlserver(src_catalog, src_schema, src_table)
elif src_type == 'postgresql':
    table_size_mb = get_table_size_postgresql(src_catalog, src_schema, src_table, jdbc_config_file=jdbc_config_file)
elif src_type == 'delta':
    table_size_mb = get_table_size_delta(src_catalog, src_schema, src_table)
else:
    raise ValueError(f'Unsupported src_type: {src_type}')

print(f'Table size MB: {table_size_mb}')

In [ ]:
lower_bound, upper_bound = get_partition_boundaries(src_catalog, src_schema, src_table, partition_col)

print(f'Upper and lower bound: {lower_bound}, {upper_bound}')

In [ ]:
# Calculate number of partitions. Minimum is 2.
num_partitions = int(table_size_mb / partition_size_mb)
num_partitions = max(num_partitions, 2)

print(f'Number of partitions: {num_partitions}')

In [ ]:
# Generate partition list
partition_list = get_partition_list(
    partition_col,
    lower_bound,
    upper_bound,
    num_partitions
)

# TODO set partitions table suffix at job level
partitions_tbl = f'{tgt_catalog}.{tgt_schema}.{tgt_table}_partitions'

# Write partitions to table
partition_df = get_partition_df(partition_list, num_partitions)
partition_df.write.option("overwriteSchema", "true").mode("overwrite").saveAsTable(partitions_tbl)

In [ ]:
# Get distinct list of batch_ids
batch_list_qry = f"""\
    select array_agg(distinct batch_id) as batch_list
    from {partitions_tbl}
"""

batch_list = spark.sql(batch_list_qry).collect()[0][0]
batch_list.sort()
print(f'batch_list: {batch_list}')

# Assign batch list to job task value. The lvl1 job iterates over the batches.
dbutils.jobs.taskValues.set(key="batch_list", value=batch_list)